In [16]:
from pynetdicom.presentation import PresentationContext
cx = PresentationContext()
cx.context_id = 1
cx.abstract_syntax = '1.2.840.10008.1.1'
cx.transfer_syntax = ['1.2.840.10008.1.2', '1.2.840.10008.1.2.4.50']

print(cx)


ID: 1
Abstract Syntax: Verification SOP Class
Transfer Syntax(es):
    =Implicit VR Little Endian
    =JPEG Baseline (Process 1)


In [22]:
from pynetdicom import build_context
from pynetdicom.sop_class import Verification
from pydicom.uid import ExplicitVRBigEndian, ExplicitVRLittleEndian, ImplicitVRLittleEndian,DeflatedExplicitVRLittleEndian


print(Verification)

cx = build_context(Verification, [ExplicitVRBigEndian, ExplicitVRLittleEndian, ImplicitVRLittleEndian, DeflatedExplicitVRLittleEndian])
print(cx._transfer_syntax)

1.2.840.10008.1.1
['1.2.840.10008.1.2.2', '1.2.840.10008.1.2.1', '1.2.840.10008.1.2', '1.2.840.10008.1.2.1.99']


In [69]:
from pynetdicom import AE, build_context, evt
from pynetdicom.sop_class import Verification
ae = AE()
ae.requested_contexts = [build_context(Verification)]
def handle_store(event):
    return 0x0000
def handle_accepted(event):
    print("Association accepted")

def handle_c_echo(event):
    print("C-ECHO request received")
    return 0x0000
handlers = [(evt.EVT_C_STORE, handle_store), (evt.EVT_ACCEPTED, handle_accepted), (evt.EVT_C_ECHO, handle_c_echo)]
connected = ae.associate('127.0.0.1', 11112, ae_title="DCM4CHEE",evt_handlers=handlers)
print(connected.is_established)





False


In [59]:
from pynetdicom import AE, build_context, StoragePresentationContexts
from pynetdicom.sop_class import CTImageStorage
ae = AE()

# ae.add_requested_context(CTImageStorage)
# ae.requested_contexts = StoragePresentationContexts
context = [
    build_context(CTImageStorage)
]
ae.requested_contexts = context

print(ae)


Application Entity PYNETDICOM

  Requested Presentation Contexts:
	CT Image Storage
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big Endian

  Supported Presentation Contexts:
	None

  ACSE timeout: 30 s
  DIMSE timeout: 30 s
  Network timeout: 60 s
  Connection timeout: None s

  Require called AE title: False

  Association(s): 0/10


In [67]:
from pynetdicom.sop_class import CTImageStorage, MRImageStorage
from pynetdicom import AE, build_context, evt, AllStoragePresentationContexts
from pydicom.uid import JPEG2000Lossless
ae = AE()
ae.supported_contexts = [build_context(CTImageStorage)]
ae.add_requested_context(MRImageStorage, JPEG2000Lossless)
ae.supported_contexts = AllStoragePresentationContexts
print(ae)


Application Entity PYNETDICOM

  Requested Presentation Contexts:
	MR Image Storage
		JPEG 2000 Image Compression (Lossless Only)

  Supported Presentation Contexts:
	Computed Radiography Image Storage
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big Endian
	Digital X-Ray Image Storage - For Presentation
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big Endian
	Digital X-Ray Image Storage - For Processing
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big Endian
	Digital Mammography X-Ray Image Storage - For Presentation
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big Endian
	Digital Mammography X-Ray Image Storage - For Processing
		Implicit VR Little Endian
		Explicit VR Little Endian
		Deflated Explicit VR Little Endian
		Explicit VR Big En

In [77]:
from pynetdicom import AE, build_context, AllStoragePresentationContexts
from pynetdicom.sop_class import UltrasoundImageStorage, MRImageStorage
from pydicom.uid import JPEG2000Lossless
ae = AE(ae_title="PACS")
context = [
    build_context(UltrasoundImageStorage,transfer_syntax=[JPEG2000Lossless])
]
print(context)
ae.requested_contexts = context
# ae.add_requested_context(UltrasoundImageStorage)
print(ae)

[Ultrasound Image Storage]

Application Entity PACS

  Requested Presentation Contexts:
	Ultrasound Image Storage
		JPEG 2000 Image Compression (Lossless Only)

  Supported Presentation Contexts:
	None

  ACSE timeout: 30 s
  DIMSE timeout: 30 s
  Network timeout: 60 s
  Connection timeout: None s

  Require called AE title: False

  Association(s): 0/10


In [85]:
from pynetdicom import AE
ae = AE()
ae.add_supported_context('1.2.840.10008.1.1')
ae.start_server(("127.0.0.1", 11112))

In [114]:
from pynetdicom import AE, build_context, evt, StoragePresentationContexts
from pynetdicom.sop_class import Verification, PatientStudyOnlyQueryRetrieveInformationModelGet


ae = AE()
ae.requested_contexts = StoragePresentationContexts[:127]
ae.add_requested_context(PatientStudyOnlyQueryRetrieveInformationModelGet)

association = ae.associate('127.0.0.1', 11112, ae_title="DCM4CHEE")
if association.is_established:
    print("Association accepted")
    association.release()
else:
    print("Association rejected")


Association accepted


In [121]:
import logging
from pynetdicom import AE, evt, debug_logger
from pynetdicom.sop_class import Verification
debug_logger()
Logger = logging.getLogger('pynetdicom')

ae = AE()
ae.add_requested_context(Verification)
def handle_connection_open(event,arg1,arg2):
    
    Logger.info("Extra args: '{}' and '{}'".format(arg1, arg2))
def handle_open(event):
    Logger.info(event)
    Logger.info(f"Event keys: {list(event.__dict__.keys())}")
    Logger.info("Open event", extra={'event': event})
handlers = [(evt.EVT_CONN_OPEN, handle_connection_open,['arg1','arg2']), (evt.EVT_CONN_OPEN, handle_open)]
association = ae.associate('127.0.0.1', 11112,ae_title="DCM4CHEE",evt_handlers=handlers)


I: Requesting Association
I: Extra args: 'arg1' and 'arg2'
I: <pynetdicom.events.Event object at 0x75187a3cd2b0>
I: Event keys: ['assoc', '_event', 'timestamp', '_hash', '_decoded', 'address']
I: Open event
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Our Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     DCM4CHEE
D: Our Max PDU Receive Size:    16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Verification SOP Class
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D: Requested Extended Negotiation: None
D: Requested Comm

E: Network timeout reached
I: Aborting Association
D: Abort Parameters:
D: =========================== OUTGOING A-ABORT PDU ===========================
D: Abort Source: DUL service-user
D: Abort Reason: (no value available)
D: ============================= END A-ABORT PDU ==============================


In [1]:
import os
from pynetdicom import AE, evt, StoragePresentationContexts
from pynetdicom.sop_class import Verification, CTImageStorage
from pydicom import dcmread

dicom_dir = "dicom"
if not os.path.exists(dicom_dir):
    os.makedirs(dicom_dir)

def handle_store(event):
    """Handle a C-STORE request and save the dataset to the dicom folder"""
    dataset = event.dataset
    filename = f"dicom/{dataset.SOPInstanceUID}.dcm"
    dataset.save_as(filename)
    print(f"Stored DICOM file: {filename}")
    return 0x0000

ae = AE()
ae.add_supported_context(Verification)
ae.add_supported_context(CTImageStorage)
ae.supported_contexts = StoragePresentationContexts[:127]
handlers = [(evt.EVT_C_STORE, handle_store)]
ae.start_server(("127.0.0.1", 11112), ae_title='STORE_SCP', evt_handlers=handlers)



In [13]:
from pynetdicom import AE, debug_logger
from pynetdicom.sop_class import Verification
# python3 -m pynetdicom echoscp 11112 -v

ae = AE()
ae.add_requested_context(Verification)
association = ae.associate('127.0.0.1',11112)
if association.is_established:
    print("Association accepted")
    status = association.send_c_echo()
    print(status)
    association.release()
else:
    print("Association rejected")






I: Requesting Association
D: Request Parameters:
D: ======================= OUTGOING A-ASSOCIATE-RQ PDU ========================
D: Our Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Our Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     ANY-SCP
D: Our Max PDU Receive Size:    16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Verification SOP Class
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D: Requested Extended Negotiation: None
D: Requested Common Extended Negotiation: None
D: Requested Asynchronous Operations Window Negotiation: None
D: Requested User Identity Negotiation: None
D: ========================== END A-ASSOCIATE

Association accepted
(0000, 0900) Status                              US: 0


In [36]:
from pynetdicom import AE, debug_logger, evt, AllStoragePresentationContexts, ALL_TRANSFER_SYNTAXES
from pynetdicom.sop_class import CTImageStorage, ModalityWorklistInformationFind
from pydicom.uid import ExplicitVRLittleEndian
# python3 -m pynetdicom storescu 127.0.0.1 11112 CTImageStorage.dcm -v -cx
debug_logger()
print(ALL_TRANSFER_SYNTAXES)

ae = AE()
# ae.add_supported_context(CTImageStorage, ExplicitVRLittleEndian)
storage_sop_classes = [
    cx.abstract_syntax for cx in AllStoragePresentationContexts
]
for uid in storage_sop_classes:
    ae.add_supported_context(uid, ALL_TRANSFER_SYNTAXES)
ae.add_supported_context(ModalityWorklistInformationFind)
def handle_store(event):
    print(event.file_meta)
    ds = event.dataset
    ds.save_as(f"dicom/{ds.SOPInstanceUID}.dcm")
    return 0x0000
def handle_find(event):
    print("Event keys:", list(event.__dict__.keys()))
    print(list(event.request.__dict__.keys()))
    return 0x0000
handles = [(evt.EVT_C_STORE, handle_store), (evt.EVT_C_FIND, handle_find)]
ae.start_server(("127.0.0.1", 11112), ae_title='STORE_SCP', evt_handlers=handles, block=True)

['1.2.840.10008.1.2', '1.2.840.10008.1.2.1', '1.2.840.10008.1.2.1.99', '1.2.840.10008.1.2.2', '1.2.840.10008.1.2.4.50', '1.2.840.10008.1.2.4.51', '1.2.840.10008.1.2.4.57', '1.2.840.10008.1.2.4.70', '1.2.840.10008.1.2.4.80', '1.2.840.10008.1.2.4.81', '1.2.840.10008.1.2.4.90', '1.2.840.10008.1.2.4.91', '1.2.840.10008.1.2.4.92', '1.2.840.10008.1.2.4.93', '1.2.840.10008.1.2.4.94', '1.2.840.10008.1.2.4.95', '1.2.840.10008.1.2.4.100', '1.2.840.10008.1.2.4.101', '1.2.840.10008.1.2.4.101.1', '1.2.840.10008.1.2.4.102', '1.2.840.10008.1.2.4.102.1', '1.2.840.10008.1.2.4.103', '1.2.840.10008.1.2.4.103.1', '1.2.840.10008.1.2.4.104', '1.2.840.10008.1.2.4.104.1', '1.2.840.10008.1.2.4.105', '1.2.840.10008.1.2.4.105.1', '1.2.840.10008.1.2.4.106', '1.2.840.10008.1.2.4.106.1', '1.2.840.10008.1.2.4.107', '1.2.840.10008.1.2.4.108', '1.2.840.10008.1.2.4.201', '1.2.840.10008.1.2.4.202', '1.2.840.10008.1.2.4.203', '1.2.840.10008.1.2.4.204', '1.2.840.10008.1.2.4.205', '1.2.840.10008.1.2.5']


D: Request Parameters:
D: ======================= INCOMING A-ASSOCIATE-RQ PDU ========================
D: Their Implementation Class UID:      1.2.826.0.1.3680043.9.3811.2.1.0
D: Their Implementation Version Name:   PYNETDICOM_210
D: Application Context Name:    1.2.840.10008.3.1.1.1
D: Calling Application Name:    PYNETDICOM
D: Called Application Name:     ANY-SCP
D: Their Max PDU Receive Size:  16382
D: Presentation Context:
D:   Context ID:        1 (Proposed)
D:     Abstract Syntax: =Modality Worklist Information Model - FIND
D:     Proposed SCP/SCU Role: Default
D:     Proposed Transfer Syntaxes:
D:       =Implicit VR Little Endian
D:       =Explicit VR Little Endian
D:       =Deflated Explicit VR Little Endian
D:       =Explicit VR Big Endian
D: Requested Extended Negotiation: None
D: Requested Common Extended Negotiation: None
D: Requested Asynchronous Operations Window Negotiation: None
D: Requested User Identity Negotiation: None
D: ========================== END A-ASSOCIATE-R

Event keys: ['assoc', '_event', 'timestamp', '_hash', '_decoded', 'request', 'context', '_is_cancelled']
['_dataset', 'OffendingElement', 'ErrorComment', '_affected_sop_class_uid', '_message_id', '_priority', '_context_id', '_dataset_path', '_dataset_file']
